# jieba中文处理

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个工具去把完整的文本中分解成粒度更细的词。   

jieba就是这样一个非常好用的中文工具，是以分词起家的，但是功能比分词要强大很多。

## 1.基本分词函数与用法

jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)

jieba.cut 方法接受三个输入参数:
- 需要分词的字符串
- cut_all 参数用来控制是否采用全模式
- HMM 参数用来控制是否使用 HMM 模型

jieba.cut_for_search 方法接受两个参数
- 需要分词的字符串
- 是否使用 HMM 模型。

该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细

In [1]:
import jieba

# 全模式
seg_list = jieba.cut('我在学习自然语言处理', cut_all = True)
print(seg_list)
print('Full Mode:' + '/'.join(seg_list))

Building prefix dict from the default dictionary ...


<generator object Tokenizer.cut at 0x10bcf3620>


Dumping model to file cache /var/folders/w2/qnnfb62x2g760j3nkh9q4ptr0000gn/T/jieba.cache
Loading model cost 0.853 seconds.
Prefix dict has been built succesfully.


Full Mode:我/在/学习/自然/自然语言/语言/处理


In [2]:
# 精确模式
seg_list = jieba.cut('我在学习自然语言处理', cut_all = False)
print('Default Mode:' + '/'.join(seg_list))

Default Mode:我/在/学习/自然语言/处理


In [3]:
# 默认是精确模式
seg_list = jieba.cut('他毕业于上海交通大学，在百度深度学习研究院进行研究') 
print(','.join(seg_list))

他,毕业,于,上海交通大学,，,在,百度,深度,学习,研究院,进行,研究


In [4]:
# 搜索引擎模式
seg_list = jieba.cut_for_search('小明硕士毕业于中国科学院计算所，后在哈佛大学深造')
print(','.join(seg_list))

小明,硕士,毕业,于,中国,科学,学院,科学院,中国科学院,计算,计算所,，,后,在,哈佛,大学,哈佛大学,深造


### jie.cut以及jieba.cut_for_search直接返回list

In [5]:
result_lcut = jieba.lcut('小明硕士毕业于中国科学院计算所，后在哈佛大学深造')
print(result_lcut)
print()
print(' '.join(result_lcut))
print(' '.join(jieba.lcut_for_search('小明硕士毕业于中国科学院计算所，后在哈佛大学深造')))

['小明', '硕士', '毕业', '于', '中国科学院', '计算所', '，', '后', '在', '哈佛大学', '深造']

小明 硕士 毕业 于 中国科学院 计算所 ， 后 在 哈佛大学 深造
小明 硕士 毕业 于 中国 科学 学院 科学院 中国科学院 计算 计算所 ， 后 在 哈佛 大学 哈佛大学 深造


### 添加用户自定义词典

很多时候我们需要针对自己的场景进行分词，会有一些领域内的专有词汇。
- 1.可以用jieba.load_userdict(file_name)加载用户字典
- 2.少量的词汇可以自己用下面方法手动添加：
    - 用 add_word(word, freq=None, tag=None) 和 del_word(word) 在程序中动态修改词典
    - 用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。

In [6]:
# 如果是动态添加词频，HMM一定要设置为False
print('/'.join(jieba.cut('如果放在旧字典中将出错。', HMM=False)))

如果/放在/旧/字典/中将/出错/。


In [7]:
jieba.suggest_freq(('中','将'), True)

494

In [8]:
print('/'.join(jieba.cut('如果放到旧字典中将出错。', HMM=False)))

如果/放到/旧/字典/中/将/出错/。


### 关键词提取

#### 基于 TF-IDF 算法的关键词抽取
import jieba.analyse
- jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
    - sentence 为待提取的文本
    - topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
    - withWeight 为是否一并返回关键词权重值，默认值为 False
    - allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [9]:
import jieba.analyse as analyse

lines = open('./files/NBA.txt').read()

print(" ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

韦少 杜兰特 全明星 全明星赛 MVP 威少 正赛 科尔 投篮 勇士 球员 斯布鲁克 更衣柜 NBA 三连庄 张卫平 西部 指导 雷霆 明星队


In [10]:
for word_weight in analyse.extract_tags(lines, topK=10, withWeight=True, allowPOS=()):
    print(word_weight)

('韦少', 0.40006111722303145)
('杜兰特', 0.3557260029055118)
('全明星', 0.30987389050216535)
('全明星赛', 0.28603743738661414)
('MVP', 0.28239608274566924)
('威少', 0.2588630758501968)
('正赛', 0.14119804137283462)
('科尔', 0.11543555039681103)
('投篮', 0.10367501793307088)
('勇士', 0.1036619725106693)


In [11]:
lines = open('./files/西游记.txt').read()
    
print(' '.join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

行者 八戒 师父 三藏 唐僧 大圣 沙僧 妖精 菩萨 和尚 那怪 那里 长老 呆子 徒弟 怎么 不知 老孙 国王 一个


#### 关于TF-IDF 算法的关键词抽取补充
- 关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径
    - 用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
        - 自定义语料库示例见<a href="https://github.com/fxsjy/jieba/blob/master/extra_dict/idf.txt.big">这里</a>
        - 用法示例见<a href="https://github.com/fxsjy/jieba/blob/master/test/extract_tags_idfpath.py">这里</a>
    - 关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径
        - 用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径
        - 自定义语料库示例见<a href="https://github.com/fxsjy/jieba/blob/master/extra_dict/stop_words.txt">这里</a>
        - 用法示例见<a href="https://github.com/fxsjy/jieba/blob/master/test/extract_tags_stop_words.py">这里</a>
- 关键词一并返回关键词权重值示例
    - 用法示例见<a href="https://github.com/fxsjy/jieba/blob/master/test/extract_tags_with_weight.py">这里</a>

##### 自定义IDF语料库和停止词语料库添加用法示例

In [12]:
import jieba
import jieba.analyse 

file_name = './files/NBA.txt'
content = open(file_name, 'r').read()

jieba.analyse.set_stop_words("./extra_dict/stop_words.txt")
jieba.analyse.set_idf_path('./extra_dict/idf.txt.big')

tags = jieba.analyse.extract_tags(content, topK=20)

print(' '.join(tags))

韦少 全明星 杜兰特 MVP 全明星赛 威少 正赛 球员 勇士 自己 一个 指导 科尔 投篮 但是 斯布鲁克 就是 成为 NBA 加入


##### 关键词一并返回关键词权重值示例

In [13]:
import jieba 
import jieba.analyse

file_name = './files/NBA.txt'
content = open(file_name, 'r').read()

tags = jieba.analyse.extract_tags(content, topK=20, withWeight=True, allowPOS=())

for tag in tags:
    print("tag: %s\t\t weight: %f" % (tag[0],tag[1]))

tag: 韦少		 weight: 0.400061
tag: 全明星		 weight: 0.309874
tag: 杜兰特		 weight: 0.305929
tag: MVP		 weight: 0.282396
tag: 全明星赛		 weight: 0.282396
tag: 威少		 weight: 0.258863
tag: 正赛		 weight: 0.141198
tag: 球员		 weight: 0.141198
tag: 勇士		 weight: 0.141198
tag: 自己		 weight: 0.141198
tag: 一个		 weight: 0.117665
tag: 指导		 weight: 0.117665
tag: 科尔		 weight: 0.115436
tag: 投篮		 weight: 0.103675
tag: 但是		 weight: 0.094132
tag: 斯布鲁克		 weight: 0.094132
tag: 就是		 weight: 0.094132
tag: 成为		 weight: 0.094132
tag: NBA		 weight: 0.070599
tag: 加入		 weight: 0.070599


#### 基于TextRank算法的关键词抽取

- jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
- jieba.analyse.TextRank() 新建自定义 TextRank 实例

<p>算法论文： <a href="http://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf">TextRank: Bringing Order into Texts</a></p>

<p>基本思想:</p>
<ul>
<li>将待抽取关键词的文本进行分词</li>
<li>以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图</li>
<li>计算图中节点的PageRank，注意是无向带权图</li>
</ul>

In [14]:
import jieba.analyse as analyse

lines = open('./files/NBA.txt').read()
print(' '.join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))


全明星赛 勇士 正赛 指导 对方 投篮 球员 没有 出现 时间 威少 认为 看来 结果 相隔 助攻 现场 三连庄 介绍 嘉宾


In [15]:
print(' '.join(analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

勇士 正赛 全明星赛 指导 投篮 玩命 时间 对方 现场 结果 球员 嘉宾 时候 全队 主持人 照片 全程 目标 快船队 肥皂剧


### 词性标注

<ul>
<li>jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。</li>
<li>标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。</li>
<li>具体的词性对照表参见<a href="http://ictclas.nlpir.org/nlpir/html/readme.htm">计算所汉语词性标记集</a></li>
</ul>

In [16]:
import jieba.posseg as pseg

words = pseg.cut('我爱自然语言处理')
for word, flag in words:
    print('%s %s'%(word, flag))

我 r
爱 v
自然语言 l
处理 v


### 并行分词


<div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p>原理：将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词，然后归并结果，从而获得分词速度的可观提升
基于 python 自带的 multiprocessing 模块，目前暂不支持 Windows</p>
<p>用法：</p>

<pre><code>jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数
jieba.disable_parallel() # 关闭并行分词模式</code></pre>
<p>实验结果：在 4 核 3.4GHz Linux 机器上，对金庸全集进行精确分词，获得了 1MB/s 的速度，是单进程版的 3.3 倍。</p>
<p>注意：并行分词仅支持默认分词器 jieba.dt 和 jieba.posseg.dt。</p>

</div>

In [17]:
import sys
import time
import jieba

# 开启并行分词模式
jieba.enable_parallel()  

content = open('./files/glory.txt','r',encoding='utf-8').read()

t1 = time.time()

words = '/'.join(jieba.cut(content))

t2 = time.time()
tm_cost = t2-t1
print('并行分词速度为%s bytes/second'%(len(content)/tm_cost))

并行分词速度为505126.65319296677 bytes/second


In [18]:
jieba.disable_parallel()

content = open('./files/glory.txt','r',encoding='utf-8').read()

t1 = time.time()

words = '/'.join(jieba.cut(content))

t2 = time.time()
tm_cost = t2-t1
print('非并行分词速度为%s bytes/second'% ( len(content)/tm_cost))

非并行分词速度为191317.61283336737 bytes/second


### Tokenize：返回词语在原文的起止位置
注意，输入参数只接受unicode

In [19]:
print('这是默认模式的Tokenize')

result = jieba.tokenize('自然语言处理非常有用')

print(result)

for tk in result:
    print('%s \t \t start: %d \t\t end:%d' % (tk[0], tk[1], tk[2]))



这是默认模式的Tokenize
<generator object Tokenizer.tokenize at 0x11568e200>
自然语言 	 	 start: 0 		 end:4
处理 	 	 start: 4 		 end:6
非常 	 	 start: 6 		 end:8
有用 	 	 start: 8 		 end:10


In [20]:
print('这是搜索模式的Tokenize')

result = jieba.tokenize('自然语言处理非常有用', mode='search')

print(result)

for tk in result:
    print('%s \t \t  start: %d \t \t end: %d' % (tk[0], tk[1], tk[2]))

这是搜索模式的Tokenize
<generator object Tokenizer.tokenize at 0x11568e3b8>
自然 	 	  start: 0 	 	 end: 2
语言 	 	  start: 2 	 	 end: 4
自然语言 	 	  start: 0 	 	 end: 4
处理 	 	  start: 4 	 	 end: 6
非常 	 	  start: 6 	 	 end: 8
有用 	 	  start: 8 	 	 end: 10


### ChineseAnalyzer for Whoosh 搜索引擎
- from jieba.analyse import ChineseAnalyzer

Whoosh是一个索引文本和搜索文本的类库，他可以为你提供搜索文本的服务，比如如果你在创建一个博客的软件，你可以用whoosh为它添加添加一个搜索功能以便用户来搜索博客的入口



In [21]:
import os
import sys
import jieba
from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser

In [22]:
analyzer = jieba.analyse.ChineseAnalyzer()
schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT(stored=True, analyzer=analyzer))

不管我们做什么应用，只要和用户输入打交道，就有一个原则--永远不要相信用户的输入数据。意味着我们要对用户输入进行严格的验证，web开发时一般输入数据都以JSON形式发送到后端API，API要对输入数据做验证。一般我都是加很多判断，各种if，导致代码很丑陋，能不能有一种方式比较优雅的验证用户数据呢？Schema就派上用场了。

Schema非常简单，也就几百行的代码，最核心的类就一个：Schema。

<a href='https://segmentfault.com/a/1190000011777230'>Python Schema使用说明</a>


In [23]:
if not os.path.exists('tmp'):
    os.mkdir('tmp')

In [24]:
ix = create_in('tmp', schema) # for create new index 
# ix = open_dir('tmp')  # for read only
writer = ix.writer()

In [25]:
writer.add_document(
    title='document1',
    path='/a', 
    content = "This is the first document we've added!")

In [26]:
writer.add_document(
    title="document2",
    path="/b",
    content="The second one 你 中文测试中文 is even more interesting! 吃水果"
)

In [27]:
writer.add_document(
    title="document3",
    path="/c",
    content="买水果然后来世博园。"
)

In [28]:
writer.add_document(
    title="document4",
    path="/c",
    content="工信处女干事每月经过下属科室都要亲口交代24口交换机等技术性器件的安装工作"
)

In [29]:
writer.add_document(
    title="document4",
    path="/c",
    content="咱俩交换一下吧。"
)

In [30]:
writer.commit()

In [31]:
searcher = ix.searcher()

In [32]:
parser = QueryParser('content', schema = ix.schema)

In [33]:
for keyword in ('水果世博园', '你', 'first', '中文', '交换机', '交换'):
    print("【关键词】的结果如下：", keyword)
    q = parser.parse(keyword)
    results = searcher.search(q)
    for hit in results:
        print(hit.highlights('content'))
        print('\n\n')

【关键词】的结果如下： 水果世博园
买<b class="match term0">水果</b>然后来<b class="match term1">世博园</b>



【关键词】的结果如下： 你
second one <b class="match term0">你</b> 中文测试中文 is even more interesting



【关键词】的结果如下： first
<b class="match term0">first</b> document we've added



【关键词】的结果如下： 中文
second one 你 <b class="match term0">中文</b>测试<b class="match term0">中文</b> is even more interesting



【关键词】的结果如下： 交换机
干事每月经过下属科室都要亲口交代24口<b class="match term0">交换机</b>等技术性器件的安装工作



【关键词】的结果如下： 交换
咱俩<b class="match term0">交换</b>一下吧



干事每月经过下属科室都要亲口交代24口<b class="match term0">交换</b>机等技术性器件的安装工作





In [34]:
for t in analyzer('我的好朋友是黎明；我爱北京天安门；IBM和Microsoft；I have a dream. This is interesting and interested me a lot'):
    print(t.text)

我
好
朋友
是
黎明
我
爱
北京
天安
天安门
ibm
microsoft
dream
interest
interest
me
lot


-----

### jieba的命令行分词

使用示例：
python -m jieba news.txt > cut_result.txt

p>命令行选项（翻译）：</p>

<pre><code>使用: python -m jieba [options] filename

结巴命令行界面。

固定参数:
  filename              输入文件

可选参数:
  -h, --help            显示此帮助信息并退出
  -d [DELIM], --delimiter [DELIM]
                        使用 DELIM 分隔词语，而不是用默认的' / '。
                        若不指定 DELIM，则使用一个空格分隔。
  -p [DELIM], --pos [DELIM]
                        启用词性标注；如果指定 DELIM，词语和词性之间
                        用它分隔，否则用 _ 分隔
  -D DICT, --dict DICT  使用 DICT 代替默认词典
  -u USER_DICT, --user-dict USER_DICT
                        使用 USER_DICT 作为附加词典，与默认词典或自定义词典配合使用
  -a, --cut-all         全模式分词（不支持词性标注）
  -n, --no-hmm          不使用隐含马尔可夫模型
  -q, --quiet           不输出载入信息到 STDERR
  -V, --version         显示版本信息并退出

如果没有指定文件名，则使用标准输入。

</code></pre>
<p><code>--help</code> 选项输出：</p>

<pre><code>$&gt; python -m jieba --help
Jieba command line interface.

positional arguments:
  filename              input file

optional arguments:
  -h, --help            show this help message and exit
  -d [DELIM], --delimiter [DELIM]
                        use DELIM instead of ' / ' for word delimiter; or a
                        space if it is used without DELIM
  -p [DELIM], --pos [DELIM]
                        enable POS tagging; if DELIM is specified, use DELIM
                        instead of '_' for POS delimiter
  -D DICT, --dict DICT  use DICT as dictionary
  -u USER_DICT, --user-dict USER_DICT
                        use USER_DICT together with the default dictionary or
                        DICT (if specified)
  -a, --cut-all         full pattern cutting (ignored with POS tagging)
  -n, --no-hmm          don't use the Hidden Markov Model
  -q, --quiet           don't print loading messages to stderr
  -V, --version         show program's version number and exit

If no filename specified, use STDIN instead.</code></pre>

</div>
</div>
</div>
</div>
</div>
</div>

</div>
    </div>
  </div>

